In [1]:
import numpy as np

In [2]:
NUM_ACTIONS = 2
GAMMA = 0.9

In [70]:
# Expected rewards for each state action pair
R = np.array([-40, -80, -160, -380, -100, -100, -100, -100])
R2 = np.array([0 ,0, 0, -500, -250, -250, -250, -250])
#R2 = np.random.normal(-200, 20, size=(8,))

In [109]:

PHI = np.eye(4)
# PHI = np.array(
#     [
#         [1.25,0],
#         [1.5,0],
#         [1.75,0],
#         [0,1]
#     ]
# )
P_0 = np.array(
    [
        [0.5, 0.5, 0, 0],
        [0, 0.5, 0.5, 0],
        [0, 0, 0.5, 0.5],
        [0.5, 0, 0, 0.5]
    ]
)
P_1 = np.concatenate((np.ones((4,1)), np.zeros((4, 3))), axis=1)
A = np.concatenate((np.eye(4), np.eye(4))) - GAMMA * np.concatenate((P_0, P_1))
NUM_STATES, NUM_FEATS = PHI.shape
#c = np.ones(NUM_STATES) / np.sum(c)
c = np.array([1, 0, 0, 0])
c, PHI, A

(array([1, 0, 0, 0]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[ 0.55, -0.45,  0.  ,  0.  ],
        [ 0.  ,  0.55, -0.45,  0.  ],
        [ 0.  ,  0.  ,  0.55, -0.45],
        [-0.45,  0.  ,  0.  ,  0.55],
        [ 0.1 ,  0.  ,  0.  ,  0.  ],
        [-0.9 ,  1.  ,  0.  ,  0.  ],
        [-0.9 ,  0.  ,  1.  ,  0.  ],
        [-0.9 ,  0.  ,  0.  ,  1.  ]]))

In [110]:
#PHI_HAT = np.eye(8)
corners = np.zeros((NUM_STATES,NUM_FEATS))
PHI_HAT = np.concatenate((np.concatenate((PHI, corners), axis=1), np.concatenate((corners, PHI), axis=1)))
R_TILDE = np.concatenate((R,R2), axis=np.newaxis).reshape(2,8).T
#R_TILDE = np.eye(8)
#R_TILDE = R.reshape(8,1)
PHI_HAT, R_TILDE

(array([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[ -40,    0],
        [ -80,    0],
        [-160,    0],
        [-380, -500],
        [-100, -250],
        [-100, -250],
        [-100, -250],
        [-100, -250]]))

In [111]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('lp')

w = m.addMVar(NUM_FEATS, lb=float('-inf'))
xi = m.addMVar(R_TILDE.shape[1], lb=0)
c = c.T @ PHI
m.setMObjective(None, c=c, constant=0.0, xc=w, sense=GRB.MINIMIZE)

m.addConstr(A @ PHI @ w >= PHI_HAT @ R_TILDE @ xi)
m.addConstr(sum(xi) == 1)
#m.addConstrs(xi[i] >= 0  for i in range(NUM_FEATS, NUM_FEATS + R_TILDE.shape[1]))

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x0f0d4671
Coefficient statistics:
  Matrix range     [1e-01, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 4 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -8.7020299e+02   1.386502e+01   0.000000e+00      0s
       2   -6.1538462e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective -6.153846154e+02


In [112]:
w_opt = np.array([v.x for v in m.getVars()])[0:NUM_FEATS]
xi = np.array([v.x for v in m.getVars()])[NUM_FEATS:]
xi

array([0.84615385, 0.15384615])

In [113]:
PHI @ w_opt

array([-615.38461538, -676.92307692, -676.92307692, -526.4957265 ])

In [114]:
np.round(A @ PHI @ w_opt - PHI_HAT @ R_TILDE @ xi)

array([ -0.,   0.,   0., 386.,  62.,   0.,  -0., 150.])

In [115]:
m.pi[0:4], m.pi[4:8]

([5.846153846153845, 1.6923076923076918, -0.0, 0.0],
 [0.0, 1.6999999999999997, 0.7615384615384614, 0.0])

In [116]:
u = np.array(m.pi[0:8]).reshape(2,4).T
u = u / u.sum(axis=1).reshape(4,1)
u

<ipython-input-116-767ff456d3b9>:2: RuntimeWarning: invalid value encountered in true_divide
  u = u / u.sum(axis=1).reshape(4,1)


array([[ 1.        ,  0.        ],
       [ 0.49886621,  0.50113379],
       [-0.        ,  1.        ],
       [        nan,         nan]])